In [2]:
from sklearn.cluster import KMeans
import numpy as np
from PIL import Image
import glob
from numpy import asarray

In [96]:
filelist = glob.glob('ORL/*.jpg')
list = []
for fname in filelist:
    pic_num = int(fname.split("\\")[-1].split(".")[0].split("_")[0])
    cluster_num = int(fname.split("\\")[1].split(".")[0].split("_")[1])
    img_arr = asarray(Image.open(fname).convert('L')).flatten()
    list.append([pic_num, cluster_num, img_arr])
    #print(img_arr)

#1, 2, 3(arr)
data = np.array(list)
data.shape, list[0]

((410, 3), [100, 10, array([100, 118, 111, ...,  21,  20,  21], dtype=uint8)])

In [97]:
out_arr1 = np.stack(data[:, 2])
out_arr1.shape

(410, 5600)

In [98]:
kmeans = KMeans(n_clusters= 41, random_state = 11).fit(np.stack(data[:, 2]))

In [99]:
output_labels = np.array(kmeans.labels_).reshape((len(kmeans.labels_), 1))
output_labels.shape, data.shape

((410, 1), (410, 3))

In [100]:
data_with_label = np.append(data, output_labels, axis=1)
data_with_label.shape, data_with_label[0].shape, data_with_label[0][2].shape, type(data_with_label[0][3])
# data_with_label = (int) pic_num, (int) cluster_num, (np arr) pic, (int) predicted_cluster of pic

((410, 4), (4,), (5600,), int)

In [114]:
def ri(data_with_label, k = 41):
    pred_cluster_dict = {} # key is label predicted by kmeans, values are pic_nums of pics that are labeled with the respected key
    for i, pred_cluster in enumerate(data_with_label[:, 3]): 
        if pred_cluster in pred_cluster_dict.keys():
          pred_cluster_dict[pred_cluster].append(data_with_label[i][0])
        else:
          pred_cluster_dict[pred_cluster] = [data_with_label[i][0]]
    
    data_label_dict = {} # key is pic_num, values are true_label(original label) of that key
    for i, data_id in enumerate(data_with_label[:, 0]): 
        data_label_dict[data_id] = data_with_label[i][1]
    
    data_pop_dict = {} # key is original cluster numnber, value is population
    for i, label in enumerate(data_with_label[:, 1]):
        if label in data_pop_dict.keys():
          data_pop_dict[label]+=1
        else:
          data_pop_dict[label] = 1
    
    TP_FP = 0
    for key in pred_cluster_dict:
        Ni = len(pred_cluster_dict[key])
        TP_FP += int((Ni * (Ni-1))/2)
        
    TP = 0
    for key in pred_cluster_dict: 
        item_count = {}
        # print(label_dict[key])
        for item in pred_cluster_dict[key]:
            if data_label_dict[item] in item_count.keys():
                item_count[data_label_dict[item]]+= 1
            else:
                item_count[data_label_dict[item]] = 1
        # print(item_count)

        for i in item_count.keys():
            same_items = item_count[i]
            TP += int(same_items * (same_items-1)/2)
            # print(TP)
    
    FP = TP_FP - TP

    FN = 0
    for data_type in data_pop_dict:
        Ni = data_pop_dict[data_type]
        TPi = 0
        for key in pred_cluster_dict:
            pop_in_curr_cluster = 0
            for data in pred_cluster_dict[key]:
                if data_label_dict[data] == data_type:
                    pop_in_curr_cluster += 1
            TPi += int(pop_in_curr_cluster * (pop_in_curr_cluster-1)/2)
        #print(TPi)
        FN += int((Ni * (Ni-1)/2) - TPi)
    
    N = 0
    for data_type in data_pop_dict:
        N += data_pop_dict[data_type]
    TN = int(N*(N-1)/2 - (FN + TP + FP))

    RI = (TP + TN) / (TP + TN + FP + FN)
    return RI

In [115]:
print(ri(data_with_label))

0.9830520603494544
